In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy import optimize
import random
np.random.seed(42)

## Misspecified Economy 1: i.i.d. growth

In [3]:
delta = 0.98
pi_0 = 0.5
theta_0 = 2
g_H = 1.2
g_L = 1
pi = 0.7

def SDF(G,theta):
    return delta*G**(-theta)

Rf = 1/(pi*SDF(g_H,theta_0)+(1-pi)*SDF(g_L,theta_0))
T = 500
data = np.random.choice([g_H, g_L], size=T, p=[pi_0, 1-pi_0])
nT = np.sum(data == g_H)

### Estimating Subjective Belief Given $\theta$

In [5]:
def lambda_objective(lam,theta):
    return -(nT/T)*np.log((1+lam*(SDF(g_H,theta)*Rf-1)))-(1-nT/T)*np.log((1+lam*(SDF(g_L,theta)*Rf-1)))

In [6]:
# estimate subjective belief if we know the true theta
theta = 2
lam_hat = optimize.fmin(lambda_objective, 1, args=(theta,) ,xtol=1e-8)
pi_hat = (1/(1+lam_hat*(SDF(g_H,theta)*Rf-1)))*(nT/T)

print(f'pi_hat={pi_hat}')

Optimization terminated successfully.
         Current function value: -0.103076
         Iterations: 32
         Function evaluations: 66
pi_hat=[0.69999999]


### Jointly Estimate $\theta$ and Subjective Belief

In [8]:
def theta_objective(theta):
    lam_hat = optimize.fmin(lambda_objective, 1, args=(theta,) ,xtol=1e-8)
    pi_0_hat = nT/T 
    pi_hat = (1/(1+lam_hat*(SDF(g_H,theta)*Rf-1)))*pi_0_hat
    return pi_0_hat*np.log(pi_0_hat/pi_hat)+(1-pi_0_hat)*np.log((1-pi_0_hat)/(1-pi_hat))

In [9]:
# estimate subjective belief if we do not know true theta
# now depend on whehter the subjective belief = objective DGP
theta_hat = optimize.fmin(theta_objective, 1 ,xtol=1e-8)
lam_hat = optimize.fmin(lambda_objective, 1, args=(theta_hat,) ,xtol=1e-8)
pi_hat = (1/(1+lam_hat*(SDF(g_H,theta)*Rf-1)))*(nT/T)

print(f'theta_hat={theta_hat}')
print(f'pi_hat={pi_hat}')

Optimization terminated successfully.
         Current function value: -1.351876
         Iterations: 44
         Function evaluations: 95
Optimization terminated successfully.
         Current function value: -0.427501
         Iterations: 37
         Function evaluations: 77
Optimization terminated successfully.
         Current function value: -0.216818
         Iterations: 35
         Function evaluations: 74
Optimization terminated successfully.
         Current function value: -0.066255
         Iterations: 30
         Function evaluations: 62
Optimization terminated successfully.
         Current function value: -0.017566
         Iterations: 29
         Function evaluations: 59
Optimization terminated successfully.
         Current function value: -0.000396
         Iterations: 31
         Function evaluations: 63
Optimization terminated successfully.
         Current function value: -0.012760
         Iterations: 32
         Function evaluations: 66
Optimization terminated suc

C:\Users\BruceQin\AppData\Local\Temp\ipykernel_2228\1633005958.py:2: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  lam_hat = optimize.fmin(lambda_objective, 1, args=(theta,) ,xtol=1e-8)


In [10]:
pi = 0.5
Rf = 1/(pi*SDF(g_H,theta_0)+(1-pi)*SDF(g_L,theta_0))

theta_hat = optimize.fmin(theta_objective, 1 ,xtol=1e-8)
lam_hat = optimize.fmin(lambda_objective, 1, args=(theta_hat,) ,xtol=1e-8)
pi_hat = (1/(1+lam_hat*(SDF(g_H,theta)*Rf-1)))*(nT/T)

print(f'theta_hat={theta_hat}')
print(f'pi_hat={pi_hat}')

Optimization terminated successfully.
         Current function value: -0.636622
         Iterations: 40
         Function evaluations: 84
Optimization terminated successfully.
         Current function value: -0.455828
         Iterations: 39
         Function evaluations: 81
Optimization terminated successfully.
         Current function value: -0.342741
         Iterations: 38
         Function evaluations: 78
Optimization terminated successfully.
         Current function value: -0.264497
         Iterations: 37
         Function evaluations: 77
Optimization terminated successfully.
         Current function value: -0.163770
         Iterations: 35
         Function evaluations: 73
Optimization terminated successfully.
         Current function value: -0.103314
         Iterations: 34
         Function evaluations: 70
Optimization terminated successfully.
         Current function value: -0.039475
         Iterations: 31
         Function evaluations: 63
Optimization terminated suc

## Misspecified Economy 2: Markov Growth

In [12]:
phi1 = 0.5
phi2 = 0.4
data2 = np.zeros(T)
data2[0]= g_H
HH = 0
HL = 0
LH = 0
LL = 0

phi1_subjective = 0.7
phi2_subjective = 0.2

for t in range(1,T):
    if data2[t-1]==g_H:
        x= np.random.binomial(n=1,p=phi1)
        if x==1:
            HH+=1
            data2[t]=g_H
        else:
            HL+=1
            data2[t]=g_L
            
    if data2[t-1]==g_L:
        x= np.random.binomial(n=1,p=phi2)
        if x==1:
            LL+=1
            data2[t]=g_L
        else:
            LH+=1
            data2[t]=g_H
            
Rf_H = 1/(phi1_subjective*SDF(g_H,theta_0)+(1-phi1_subjective)*SDF(g_L,theta_0))
Rf_L = 1/(phi2_subjective*SDF(g_L,theta_0)+(1-phi2_subjective)*SDF(g_H,theta_0))

In [13]:
def lambda_objective_2(lam,theta):
    return -(HH/(HH+HL))*np.log((1+lam*(SDF(g_H,theta)*Rf_H-1)))-(HL/(HL+HH))*np.log((1+lam*(SDF(g_L,theta)*Rf_H-1)))
    -(LL/(LL+LH))*np.log((1+lam*(SDF(g_L,theta)*Rf_L-1)))-(LH/(LL+LH))*np.log((1+lam*(SDF(g_H,theta)*Rf_L-1)))

In [14]:
theta = 2
lam_hat = optimize.fmin(lambda_objective_2, 1, args=(theta,) ,xtol=1e-8)
phi1_subjective_hat = (1/(1+lam_hat*(SDF(g_H,theta)*Rf_H-1)))*(HH/(HH+HL))
phi2_subjective_hat = (1/(1+lam_hat*(SDF(g_L,theta)*Rf_L-1)))*(LL/(LL+LH))

print(phi1_subjective_hat, phi2_subjective_hat)

Optimization terminated successfully.
         Current function value: -0.091824
         Iterations: 32
         Function evaluations: 66
[0.7] [0.20608416]


## Model I: i.i.d. log-normal consumption growth

### Well-specified Case

In [162]:
mu = 0.48
sigma = 0.51
theta_0 = 10

In [164]:
def data_generator(chi_mu, chi_sigma):
    G = np.zeros(T)
    Rm = np.zeros(T)
    Rf = np.zeros(T)
    Z = 1/(np.exp((1-theta_0)*(1-chi_mu)*mu+(1-theta_0)**2*(1+chi_sigma)**2*sigma**2/2)/delta-1)
    
    for t in range(T):
        log_G = np.random.normal(mu, sigma)
        G[t] = np.exp(log_G)
        Rm[t] = G[t]*(Z+1)/Z
        Rf[t] = 1/(delta*np.exp(-theta_0*(1-chi_mu)*mu+theta_0**2*(1+chi_sigma)**2*sigma**2/2))
        
    return G, Rm, Rf

In [166]:
def epanechnikov_kernel(u):
    return (3/4) * (1 - u**2) * (np.abs(u) <= 1)

In [168]:
def kernel_estimator(x):
    P_0_hat = np.zeros((T,T))
    b_T = 4 * sigma
    for i in range(T):
        sum_i=0
        for j in range(T):
            sum_i+= epanechnikov_kernel((x[i]-x[j])/b_T)
        for j in range(T):
            P_0_hat[i,j] = epanechnikov_kernel((x[i]-x[j])/b_T)/sum_i
            
    return P_0_hat

In [170]:
def psi(x): #smooth transformation suggested by Owen
    v=1/T
    if x>v:
        return np.log(x)
    else:
        return np.log(v)-3/2+2*x/v-(x/v)**2/2

In [172]:
def lambda_objective_3 (lam, theta, P_0_hat):
    sum = 0
    for j in range(T):
        moment_f = lam[0] * (SDF(G[j], theta)*Rf[j]-1)
        moment_m = lam[1] * (SDF(G[j], theta)*Rm[j]-1)
        sum += P_0_hat[j] * psi(1+moment_f+moment_m)

    return -sum

In [174]:
def subjective_belief_estimator(theta, P_0_hat):
    P_hat =  np.zeros((T,T))
    for i in range(T):
        lam_hat_i = optimize.fmin(lambda_objective_3, (1,1), args=(theta,P_0_hat[i,:]) ,xtol=1e-8)
        print(i)
        sump = 0
        for j in range(T):
            moment_f = lam_hat_i[0] * (SDF(G[j], theta)*Rf[j]-1)
            moment_m = lam_hat_i[1] * (SDF(G[j], theta)*Rm[j]-1)
            P_hat[i,j] = P_0_hat[i,j]/(1+moment_f+moment_m)
            sump += P_hat[i,j]

        for j in range(T):
            P_hat[i,j] = P_hat[i,j]/sump
    
    return P_hat

In [176]:
mu_estimator = np.zeros(T)
sigma_estimator = np.zeros(T)

G, Rm, Rf = data_generator(0,0)
P_0_hat = kernel_estimator(np.log(G))
#P_hat = subjective_belief_estimator(10, P_0_hat)

In [177]:
#for i in range(T):
#    mu_estimator[i] = np.sum(np.log(G)*P_hat[i,])
 #   sigma_estimator[i] = np.sum ((np.log(G)-mu_estimator[i])**2*P_hat[i,:])

#print(np.mean(mu_estimator), np.mean(sigma_estimator))

In [180]:
np.mean(np.log(G))

0.45893066186402004

In [182]:
np.var(np.log(G))

0.245479717012229

In [184]:
for i in range(T):
    mu_estimator[i] = np.sum(np.log(G)*P_0_hat[i,])
    sigma_estimator[i] = np.sum ((np.log(G)-mu_estimator[i])**2*P_0_hat[i,:])

print(np.mean(mu_estimator), np.mean(sigma_estimator))

0.45864375720007 0.20629512400054742
